In [1]:
import torch

C:\Users\Lin Borong\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
t_c = [0.5,14.0,15.0,28.0,11.0,8.0,3.0,-4.0,6.0,13.0,21.0]
t_u = [35.7,55.9,58.2,81.9,56.3,48.9,33.9,21.8,48.4,60.4,68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [3]:
def loss_fn(t_p,t_c):
    squared_diffs = (t_p-t_c)**2
    return squared_diffs.mean()

In [4]:
def model(t_u, w,b):
    return w*t_u+b

In [5]:
w = torch.ones(())
b = torch.zeros(())
t_p = model(t_u,w,b)

In [6]:
t_p

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000, 21.8000,
        48.4000, 60.4000, 68.4000])

In [7]:
loss_fn(t_p,t_c)

tensor(1763.8848)

In [8]:
params = torch.tensor([1.0,0.0],requires_grad = True)

In [9]:
loss = loss_fn(model(t_u*0.1,*params),t_c)
loss.backward()
params.grad

tensor([-77.6140, -10.6400])

In [10]:
def training_loop(n_epochs,learning_rate,params,t_u,t_c):
    for i in range(1,n_epochs+1):
        if params.grad is not None:
            params.grad.zero_()
        t_p = model(t_u,*params)
        loss = loss_fn(t_p,t_c)
        loss.backward()
        with torch.no_grad():
            params -= learning_rate * params.grad
            #params = params-learning_rate*params.grad
        if i%500==0:
            print('Epoch %d, Loss %f' % (i,float(loss)))
    return params

In [11]:
training_loop(
    n_epochs=5000,
    learning_rate=1e-2,
    params = torch.tensor([1.0, 0.0], requires_grad=True),
    t_u = t_u*0.1,
    t_c = t_c
)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

# optimizer

In [12]:
import torch.optim as optim

In [13]:
dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'NAdam',
 'Optimizer',
 'RAdam',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional',
 '_multi_tensor',
 'lr_scheduler',
 'swa_utils']

In [14]:
params = torch.tensor([1.0,0.0],requires_grad = True)
learning_rate = 1e-2
optimizer = optim.SGD([params],lr=learning_rate)
t_p = model(t_u*0.1,*params)
loss = loss_fn(t_p,t_c)
optimizer.zero_grad()
loss.backward()
optimizer.step()
params

tensor([1.7761, 0.1064], requires_grad=True)

In [15]:
def training_loop(t_u,t_c,optimizer,n_epoch,params):
    for i in range(1,n_epoch+1):
        t_p = model(t_u*0.1,*params)
        loss = loss_fn(t_p,t_c)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i%500==0:
            print('Epoch %d, Loss %f' % (i,float(loss)))
    return params

In [16]:
params = torch.tensor([1.0,0.0],requires_grad = True)
optimizer = optim.Adam([params],lr=1e-1)
training_loop(t_u,t_c,optimizer,n_epoch=5000,params=params)
params

Epoch 500, Loss 2.962302
Epoch 1000, Loss 2.927647
Epoch 1500, Loss 2.927645
Epoch 2000, Loss 2.927646
Epoch 2500, Loss 2.927646
Epoch 3000, Loss 2.927646
Epoch 3500, Loss 2.927646
Epoch 4000, Loss 2.927646
Epoch 4500, Loss 2.927646
Epoch 5000, Loss 2.927645


tensor([  5.3677, -17.3048], requires_grad=True)

In [17]:
n_samples = t_u.shape[0]
n_val = int(0.2*n_samples)
shuffled_indices = torch.randperm(n_samples)
training_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
training_indices,val_indices

(tensor([ 6,  5,  9,  2,  1,  4,  8, 10,  7]), tensor([3, 0]))

In [18]:
train_t_un = t_u[training_indices]*0.1
train_t_c = t_c[training_indices]
val_t_un = t_u[val_indices]*0.1
val_t_c = t_c[val_indices]

In [44]:
def training_loop(n_epochs, optimizer, params, training_t_u, training_t_c,val_t_u,val_t_c,loss_fn,model):
    for epoch in range(1,n_epochs+1):
        train_t_p = model(train_t_un)
        train_loss = loss_fn(train_t_p,train_t_c)
        with torch.no_grad():
            val_t_p = model(val_t_u)
            val_loss = loss_fn(val_t_p,val_t_c)
            assert val_loss.requires_grad == False
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        if epoch<=3 or epoch%500==0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                 f"Validation loss{val_loss.item():.4f}")

In [36]:
params = torch.tensor([1.0,0.0],requires_grad = True)
learning_rate = 1e-2
optimizer = optim.SGD([params],lr=learning_rate)

In [37]:
training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    params=params,
    training_t_u=train_t_un,
    val_t_u=val_t_un,
    training_t_c = train_t_c,
    val_t_c = val_t_c,
    loss_fn=loss_fn,
    model=model
)

TypeError: model() missing 2 required positional arguments: 'w' and 'b'

# pich6

In [22]:
import torch.nn as nn

In [24]:
linear_model = nn.Linear(1,1)
linear_model.weight

Parameter containing:
tensor([[0.3168]], requires_grad=True)

In [25]:
x = torch.ones(1)
linear_model(x)

tensor([-0.1998], grad_fn=<AddBackward0>)

In [39]:
t_c = t_c.clone().unsqueeze(1)
t_u = t_u.clone().unsqueeze(1)
n_samples = t_u.shape[0]
n_val = int(0.2*n_samples)
shuffled_indices = torch.randperm(n_samples)
training_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
training_indices,val_indices
train_t_un = t_u[training_indices]*0.1
train_t_c = t_c[training_indices]
val_t_un = t_u[val_indices]*0.1
val_t_c = t_c[val_indices]

In [40]:
optimizer = optim.SGD(linear_model.parameters(),lr=1e-2)

In [45]:
training_loop(
    n_epochs=3000,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    training_t_c=  train_t_c,
    training_t_u = train_t_un,
    val_t_c = val_t_c,
    val_t_u = val_t_un,
    optimizer = optimizer,
    params = linear_model.parameters()
)

Epoch 1, Training loss 131.6745,Validation loss268.4377
Epoch 2, Training loss 50.2536,Validation loss94.3634
Epoch 3, Training loss 34.6499,Validation loss46.7613
Epoch 500, Training loss 6.9076,Validation loss8.6666
Epoch 1000, Training loss 3.4953,Validation loss5.4035
Epoch 1500, Training loss 3.0187,Validation loss4.4230
Epoch 2000, Training loss 2.9521,Validation loss4.0900
Epoch 2500, Training loss 2.9428,Validation loss3.9702
Epoch 3000, Training loss 2.9415,Validation loss3.9261


In [46]:
seq_model = nn.Sequential(
    nn.Linear(1,13),
    nn.Tanh(),
    nn.Linear(13,1)
)
seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [48]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

In [50]:
for name,param in seq_model.named_parameters():
    print(name, param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [57]:
from collections import OrderedDict
seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear',nn.Linear(1,8)),
    ("hidden_Active",nn.Tanh()),
    ('output_linear',nn.Linear(8,1))
]
)
)
seq_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=8, bias=True)
  (hidden_Active): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [62]:
optimizer = optim.SGD(seq_model.parameters(), lr = 1e-3)
training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    model=seq_model,
    training_t_c=train_t_c,
    training_t_u = train_t_un,
    val_t_c=val_t_c,
    val_t_u = val_t_un,
    loss_fn = loss_fn,
    params = seq_model.parameters()
)

Epoch 1, Training loss 154.5241,Validation loss310.9136
Epoch 2, Training loss 151.8595,Validation loss305.6057
Epoch 3, Training loss 149.2706,Validation loss300.4181
Epoch 500, Training loss 16.6472,Validation loss20.6963
Epoch 1000, Training loss 6.1230,Validation loss5.5394
Epoch 1500, Training loss 3.4950,Validation loss3.5429
Epoch 2000, Training loss 3.6062,Validation loss0.2871
Epoch 2500, Training loss 2.4679,Validation loss0.9835
Epoch 3000, Training loss 1.9445,Validation loss1.8347
Epoch 3500, Training loss 1.7181,Validation loss2.7383
Epoch 4000, Training loss 1.6378,Validation loss3.3881
Epoch 4500, Training loss 1.5999,Validation loss3.5452
Epoch 5000, Training loss 1.5749,Validation loss3.6323
